In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import json

In [11]:
# 1. Extract time-invariant vars (locations, adm criteria, tuition) from the most recent year

'''
    We will include all the universities appearing in our dataset of the following year: 2016 - 2019
    and the data in the most recent year will be selected.
    
    E.g., universities that appeared in 2019 will have their data in 2019 chosen; 
          those that didn't appear in 2019 but appeared in 2018 will also be included with their 2018 records.
    
'''

col_geo = ['UNITID', 'INSTNM', 'ADDR', 'CITY', 'STABBR', 'LONGITUD', 'LATITUDE']
col_adm = ['UNITID'] + [f'ADMCON{n}' for n in range(1, 10)]
col_fee = ['UNITID', 'TUITION2', 'TUITION3']

df_lists = []
for i in range(2016, 2020):
    geo = pd.read_csv(f"./University_data/hd{i}.csv", encoding='ISO-8859-1').loc[:, col_geo]
    adm = pd.read_csv(f"./University_data/adm{i}.csv").loc[:, col_adm]
    fee = pd.read_csv(f"./University_data/ic{i}_ay.csv").loc[:, col_fee].replace('.', np.nan).dropna().astype(int)
    geo['YEAR'] = i
    adm['YEAR'] = i
    fee['YEAR'] = i
    df_lists.append([geo, adm, fee])

def append_dfs(pos):
    df = reduce(lambda top, bottom: top.append(bottom), [df_list[pos] for df_list in df_lists])
    return df.sort_values(['UNITID', 'YEAR']).groupby('UNITID').apply(lambda df: df.iloc[-1])

df_geo = append_dfs(0)
df_admc = append_dfs(1)
df_fee = append_dfs(2)

In [12]:
# 2. Calculate the average admission rates (disaggregated by gender) in the past 10 years

col_rate = ["UNITID", "APPLCN", "ADMSSN", "APPLCNM", "ADMSSNM", "APPLCNW", "ADMSSNW"]
df_admr = pd.DataFrame()
for i in range(2010, 2020):
    admr = pd.read_csv(f"./University_data/adm{i}.csv")
    admr = admr.loc[:, col_rate].replace(".", np.nan).astype(float)
    admr["ADMR"] = admr["ADMSSN"] / admr["APPLCN"]
    admr["ADMRM"] = admr["ADMSSNM"] / admr["APPLCNM"]
    admr["ADMRW"] = admr["ADMSSNW"] / admr["APPLCNW"]
    df_admr = df_admr.append(admr)
    
df_admr = df_admr.groupby("UNITID")[["ADMR", "ADMRM", "ADMRW"]].mean().dropna().round(4)

In [13]:
# 3. Average the parameters (mean, SD) of SAT/ACT scores in the past 10 years (assuming normal distribution)

col_sco = ["SATVR25", "SATVR75", "SATMT25", "SATMT75", 
           "ACTCM25", "ACTCM75", "ACTEN25", "ACTEN75", "ACTMT25", "ACTMT75"]
df_adms = pd.DataFrame()
for i in range(2010, 2020):
    adms = pd.read_csv(f"./University_data/adm{i}.csv", index_col="UNITID")
    adms.columns = [col.strip() for col in adms.columns]
    adms = adms.loc[:, col_sco].replace(".", np.nan).astype(float)
    adms["YEAR"] = i
    for col in ["SATVR", "SATMT", "ACTCM", "ACTEN", "ACTMT"]:
        adms[f"{col}M"] = adms[f"{col}25"] + (adms[f"{col}75"] - adms[f"{col}25"]) / 2
        adms[f"{col}SD"] = (adms[f"{col}75"] - adms[f"{col}M"]) / 0.675  #z-score for 75% percentile
    df_adms = df_adms.append(adms)
    
df_adms = df_adms.iloc[:, 10:].groupby('UNITID').mean().dropna().round(4)

In [14]:
# 4. Merge
dfls = [df_geo.iloc[:, 1:-1], df_fee.iloc[:, 1:-1], df_admc.iloc[:, 1:-1], df_admr, df_adms.iloc[:, 1:]]
final = reduce(lambda l, r: pd.merge(l, r, how='inner', left_index=True, right_index=True), dfls).dropna()

In [15]:
# Check the school with annual tuition more than 50000
final[final["TUITION3"] > 50000].drop_duplicates(subset="INSTNM")

,INSTNM,ADDR,CITY,STABBR,LONGITUD,LATITUDE,TUITION2,TUITION3,ADMCON1,ADMCON2,...,SATVRM,SATVRSD,SATMTM,SATMTSD,ACTCMM,ACTCMSD,ACTENM,ACTENSD,ACTMTM,ACTMTSD
UNITID,,,,,,,,,,,,,,,,,,,,,
110370,California College of the Arts,1111 Eighth Street,San Francisco,CA,-122.399451,37.767485,50592,50592,1,3,...,520.0000,103.7037,540.0000,103.7037,23.0000,5.9259,22.0000,4.4444,23.0000,7.4074
110404,California Institute of Technology,1200 E California Blvd,Pasadena,CA,-118.125878,34.137349,52506,52506,2,2,...,757.0000,47.4074,786.5000,20.0000,34.6000,1.0370,34.6000,1.1852,35.2500,1.1111
111948,Chapman University,One University Dr.,Orange,CA,-117.852518,33.793020,54540,54540,1,2,...,609.5000,67.4074,617.0000,72.5926,27.2000,3.5556,28.3000,5.1852,26.4000,3.5556
112260,Claremont McKenna College,500 E 9th St,Claremont,CA,-117.711188,34.102154,56190,56190,2,2,...,697.0000,62.2222,719.0000,66.6667,31.5500,2.4444,32.1000,3.1111,31.0000,3.8519
115409,Harvey Mudd College,301 Platt Blvd.,Claremont,CA,-117.709837,34.106515,58359,58359,5,5,...,728.4000,57.1852,772.5000,40.7407,33.8500,1.7037,33.9000,2.0741,34.3000,1.4815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238333,Beloit College,700 College St,Beloit,WI,-89.029285,42.504022,51050,51050,1,5,...,621.4286,101.5873,611.7857,96.8254,26.7857,4.5503,27.7857,5.8201,24.7857,3.9153
243744,Stanford University,,Stanford,CA,-122.167359,37.429434,52857,52857,2,3,...,728.5000,65.9259,748.0000,65.1852,32.8500,2.4444,33.7500,2.2963,32.6000,3.5556
247649,Landmark College,19 River Rd South,Putney,VT,-72.512549,42.977172,59100,59100,2,3,...,565.0000,111.1111,492.5000,144.4444,21.0000,4.4444,20.0000,4.4444,20.5000,6.6667


In [16]:
# Check info by specific schools
final[final["INSTNM"] == "University of Pennsylvania"]

,INSTNM,ADDR,CITY,STABBR,LONGITUD,LATITUDE,TUITION2,TUITION3,ADMCON1,ADMCON2,...,SATVRM,SATVRSD,SATMTM,SATMTSD,ACTCMM,ACTCMSD,ACTENM,ACTENSD,ACTMTM,ACTMTSD
UNITID,,,,,,,,,,,,,,,,,,,,,
215062,University of Pennsylvania,34th & Spruce Street,Philadelphia,PA,-75.19391,39.950929,51156,51156,1,3,...,719.0,62.2222,746.0,60.7407,32.75,2.4444,33.55,2.4444,32.3,3.4074


In [8]:
# Export 
final.to_csv("./University_data/merged_data.csv")

In [17]:
# 5. University ranking

with open("uni_name.json") as datafile:
    rename = json.load(datafile)
    
rank = pd.read_csv("./University_data/cwurData.csv").rename(columns={'institution':'INSTNM','national_rank':'rank'})
rank = rank.drop_duplicates(subset='INSTNM').loc[rank['country'] == 'USA', ['INSTNM', 'rank']]
rank['INSTNM'].replace({", ": "-", '–': '-'}, inplace=True, regex=True)
rank['INSTNM'].replace(rename, inplace=True)

final_rank = pd.merge(final.reset_index(), rank, on='INSTNM', how='left').sort_values('rank')
final_rank['rank'].fillna(9999, inplace=True)

In [18]:
final_rank[final_rank['ADDR'] == " "]['INSTNM'].drop_duplicates().tolist()

['Stanford University',
 'University of Maryland-College Park',
 'Michigan State University',
 'Baylor University',
 'Auburn University',
 'Hiram College',
 'Abilene Christian University']

In [19]:
# handle some empty address
final_rank.loc[final_rank['INSTNM'] == 'Stanford University','ADDR'] = "450 Serra Mall"
final_rank.loc[final_rank['INSTNM'] == 'University of Florida','ADDR'] = "Tigert Hall"
final_rank.loc[final_rank['INSTNM'] == 'Michigan State University','ADDR'] = "220 Trowbridge Rd"
final_rank.loc[final_rank['INSTNM'] == 'Baylor University','ADDR'] = "500 Speight Ave"
final_rank.loc[final_rank['INSTNM'] == 'Auburn University','ADDR']= "1161 West Samford Avenue"
final_rank.loc[final_rank['INSTNM'] == 'University of Maryland-College Park','ADDR'] = "College Park"
final_rank.loc[final_rank['INSTNM'] == 'Hiram College','ADDR'] = "11715 Garfield Road"
final_rank.loc[final_rank['INSTNM'] == 'Abilene Christian University','ADDR'] = "1600 Campus Ct"

In [20]:
# Export data with rankings
final_rank.set_index('UNITID').to_csv("./University_data/ranked_merged_data.csv")

In [21]:
# Export data with supplimentary information
sup = pd.read_excel("./University_data/suppliment.xlsx", index_col='UNITID').iloc[:, 2:]
sup['image_url'].fillna('https://imgur.com/tH33XHm', inplace=True)
sup.fillna(9999, inplace=True)
final_sup = pd.merge(final_rank.set_index('UNITID'), sup, how='left', left_index=True, right_index=True)
final_sup.to_csv('./University_data/ranked_merged_data_sup.csv')